### 목표 설정
- 네이버 영화 리뷰 데이터셋을 이용해서 긍정/부정 분류기를 만들어보자.
- TF-IDF 방법을 적용시켜보자.
- Konlpy 한국어 형태소 분석기를 활용해보자
- 단어별 긍/부정 정보를 시각화해서 알아보자

In [49]:
# 라이브러리 불러오기
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [50]:
#드라이브 경로 변경
!pwd

/content/drive/My Drive/인사교/DL/언어지능


In [51]:
# 드라이브 경로 바꾸기
%cd /content/drive/MyDrive/인사교/DL/언어지능

/content/drive/MyDrive/인사교/DL/언어지능


In [52]:
# 데이터 로딩
train =  pd.read_csv('./data/ratings_train.csv')
test = pd.read_csv('./data/ratings_test.csv')

In [53]:
train.head(3)

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0


In [54]:
test.head(3)

,id,document,label
0,6270596,굳 ㅋ,1
1,9274899,GDNTOPCLASSINTHECLUB,0
2,8544678,뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아,0


In [55]:
# 결측치 있음
train.info(), test.info()
# train - document : 결측치 5
# test - document : 결측치 3

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150000 entries, 0 to 149999
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   id        150000 non-null  int64 
 1   document  149995 non-null  object
 2   label     150000 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.4+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        50000 non-null  int64 
 1   document  49997 non-null  object
 2   label     50000 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 1.1+ MB


(None, None)

In [56]:
# 결측치 제거
train.dropna(inplace = True)
test.dropna(inplace = True)

In [57]:
train.info(), test.info()
# 결측치 제거 완료

<class 'pandas.core.frame.DataFrame'>
Index: 149995 entries, 0 to 149999
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   id        149995 non-null  int64 
 1   document  149995 non-null  object
 2   label     149995 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 4.6+ MB
<class 'pandas.core.frame.DataFrame'>
Index: 49997 entries, 0 to 49999
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        49997 non-null  int64 
 1   document  49997 non-null  object
 2   label     49997 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 1.5+ MB


(None, None)

In [58]:
# train, test 분리
# X : document / y : label
X_train = train['document']
X_test = test['document']
y_train = train['label']
y_test = test['label']

In [59]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((149995,), (49997,), (149995,), (49997,))

In [60]:
# 단어사전 만들기
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
# TF-IDF : 문서 내부에 중요한 단어 순으로 토큰화 진행
# CounterVertorizer : 문서 내부에 단어의 빈도수로 토큰화

In [61]:
# 테스트용 데이터 뽑아보기
X_train[0:3]

,document
0,아 더빙.. 진짜 짜증나네요 목소리
1,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나
2,너무재밓었다그래서보는것을추천한다


In [62]:
tfidf = TfidfVectorizer()
# 단어사전 구축
tfidf.fit(X_train[0:3])
# 단어사전 출력
tfidf.vocabulary_

{'더빙': 2,
 '진짜': 6,
 '짜증나네요': 7,
 '목소리': 3,
 '포스터보고': 9,
 '초딩영화줄': 8,
 '오버연기조차': 5,
 '가볍지': 0,
 '않구나': 4,
 '너무재밓었다그래서보는것을추천한다': 1}

- TF-IDF는 기본적으로 어절단위로 끊어주는 특성을 가지고 있다. 한국어 특성상 파악하기 어려운 부분이 있다.
- 한국어 특성을 적용해서 형태소 분석으로 진행

In [63]:
# 설치
# 한국어 형태소 분석기 konlpy 이용해보기
!pip install konlpy

In [64]:
# kkma 불러오기
# 1. konlpy의 kkma 임포트
from konlpy.tag import Kkma

In [65]:
# kkma 객체 생성
kkma = Kkma()

### Kkma 함수 정리
- nouns() : 명사 추출 함수
- morphs() : 형태소 추출 함수
- pos() : 형태소 추출 + 품사 태그 결합

In [66]:
# 명사 추출
kkma.nouns(X_train[0])

['더빙', '목소리']

In [67]:
# 형태소 추출
# 자연어 처리중 주로 사용되는 품사는 동사 / 형용사이다.
kkma.morphs(X_train[0])

['아', '아', '더빙', '..', '진짜', '짜증나', '네요', '목소리']

In [68]:
# 형태소 추출 + 품사태깅
kkma.pos(X_train[0])

[('아', 'VV'),
 ('아', 'ECS'),
 ('더빙', 'NNG'),
 ('..', 'SW'),
 ('진짜', 'MAG'),
 ('짜증나', 'VV'),
 ('네요', 'EFN'),
 ('목소리', 'NNG')]

In [69]:
# 품사 태그 정보 확인하기
kkma.tagset

{'EC': '연결 어미',
 'ECD': '의존적 연결 어미',
 'ECE': '대등 연결 어미',
 'ECS': '보조적 연결 어미',
 'EF': '종결 어미',
 'EFA': '청유형 종결 어미',
 'EFI': '감탄형 종결 어미',
 'EFN': '평서형 종결 어미',
 'EFO': '명령형 종결 어미',
 'EFQ': '의문형 종결 어미',
 'EFR': '존칭형 종결 어미',
 'EP': '선어말 어미',
 'EPH': '존칭 선어말 어미',
 'EPP': '공손 선어말 어미',
 'EPT': '시제 선어말 어미',
 'ET': '전성 어미',
 'ETD': '관형형 전성 어미',
 'ETN': '명사형 전성 어미',
 'IC': '감탄사',
 'JC': '접속 조사',
 'JK': '조사',
 'JKC': '보격 조사',
 'JKG': '관형격 조사',
 'JKI': '호격 조사',
 'JKM': '부사격 조사',
 'JKO': '목적격 조사',
 'JKQ': '인용격 조사',
 'JKS': '주격 조사',
 'JX': '보조사',
 'MA': '부사',
 'MAC': '접속 부사',
 'MAG': '일반 부사',
 'MD': '관형사',
 'MDN': '수 관형사',
 'MDT': '일반 관형사',
 'NN': '명사',
 'NNB': '일반 의존 명사',
 'NNG': '보통명사',
 'NNM': '단위 의존 명사',
 'NNP': '고유명사',
 'NP': '대명사',
 'NR': '수사',
 'OH': '한자',
 'OL': '외국어',
 'ON': '숫자',
 'SE': '줄임표',
 'SF': '마침표, 물음표, 느낌표',
 'SO': '붙임표(물결,숨김,빠짐)',
 'SP': '쉼표,가운뎃점,콜론,빗금',
 'SS': '따옴표,괄호표,줄표',
 'SW': '기타기호 (논리수학기호,화폐기호)',
 'UN': '명사추정범주',
 'VA': '형용사',
 'VC': '지정사',
 'VCN': "부정 지정사, 형용사 '아니다'",
 'VC

### 감성 분석에서는 객체의 동작이나 상태를 나타내는 동사와 형용사를 많이 사용한다.(기억해두면 좋을듯)
- 형용사 : VA, 동사 : VV, 보통명사 : NNG

1. konlpy에서 원하는 품사만 뽑아서 형태소화 시키는 형태소 분석을 진행
2. 위에 기능을 TF-IDF Vectorizer에 결합해서 문장에서 형용사, 동사, 명사 태그를 가진 단어만 뽑아보자

In [70]:
#  더미데이터 만들기
data = '먹는다, 먹다, 먹었다, 이쁘다, 아름답다, 사진, 모자, 다이어리'

# 데이터 프레임 만들기
d= pd.DataFrame(kkma.pos(data), columns = ['morphs', 'tag'])

# 인덱스르 tag로 설정하기
d.set_index(d['tag'], inplace = True)

# 불리언 인덱싱(동사, 형용사, 명사)
# d[(d['tag'] == 'VV') | (d['tag'] == 'VA') | (d['tag'] == 'NNG')]

# 교집합 기능 사용해보기
d.loc[d.index.intersection(['VV', 'VA', 'NNG'])]

,morphs,tag
tag,,
VV,먹,VV
VV,먹,VV
VV,먹,VV
VA,이쁘,VA
VA,아름답,VA
NNG,사진,NNG
NNG,모자,NNG
NNG,다이어리,NNG


In [76]:
def myTokenizer(text):
  d = pd.DataFrame(kkma.pos(text), columns = ['morphs', 'tag'])
  d.set_index(d['tag'], inplace = True)

  # Check if any of the desired tags exist in the index
  if any(tag in d.index for tag in ['VV', 'VA', 'NNG']):
    # Return the 'morphs' values for the intersection of desired tags
    return d.loc[d.index.intersection(['VV', 'VA', 'NNG']),'morphs'].values
  else:
    # Return an empty list instead of None to avoid the TypeError
    return [] # Return an empty list if no relevant tags are found

In [72]:
# TF - IDF 백터라이저에 연결해주기
tfidf_pos = TfidfVectorizer(tokenizer = myTokenizer)

# 테스트 데이터로 실험 해보기
tfidf_pos.fit(X_train[0:3])

# 단어사전 출력
tfidf_pos.vocabulary_

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


{'아': 5,
 '짜증나': 12,
 '더빙': 1,
 '목소리': 2,
 '흠': 15,
 '포스터': 14,
 '보고': 4,
 '영화': 8,
 '줄': 11,
 '오버': 9,
 '연기': 7,
 '가볍': 0,
 '재': 10,
 '추천': 13,
 '어': 6,
 '보': 3}

In [77]:
# 실전 데이터를 이용해서 단어사전 구축해보기
# 10000개의 데이터를 사용해서 단어사전 구축을 진행()
final_tfidf = TfidfVectorizer(tokenizer = myTokenizer)
final_tfidf.fit(X_train[0:10000])

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


TfidfVectorizer(tokenizer=<function myTokenizer at 0x7849b608c310>)

In [81]:
len(final_tfidf.vocabulary_)

8748

In [82]:
# 실제 문장에 단어를 토큰값으로 변환 해주는 작업 진행 : 약 8분정도 소요
X_train_token = final_tfidf.transform(X_train[0 : 10000])
X_test_token = final_tfidf.transform(X_test[0 : 10000])

In [83]:
# 간단한 머신러닝 모델로 텍스트 감성 분석
# 딥러닝 모델로 교체 가능
from sklearn.linear_model import LogisticRegression # 로지스틱 회귀
# 선형 분류(로지스틱 회귀)를 사용한 이유
# 텍스트 전처리에서 시간이 오래걸린다. 원활한 서비스를 제공하기 위해서 상대적으로 속도가 빠른 모델

In [84]:
# 모델 생성
logi = LogisticRegression()

In [85]:
X_train_token.shape, y_train.shape

((10000, 8748), (149995,))

In [86]:
# 모델 학습
logi.fit(X_train_token, y_train[:10000])

LogisticRegression()

In [87]:
logi.score(X_test_token, y_test[0:10000])

0.7838

In [91]:
# logi 활용
# predict_proba()
target_name = ['부정', '긍정']
review = ['오랜만에 볼만한 한국 영화였어요!']
vect_review = final_tfidf.transform(review)
pre = logi.predict(vect_review)
print(f'{review[0]}의 문장은{logi.predict_proba(vect_review).max() * 100:.2f}%로 {target_name[pre[0]]} 리뷰입니다.')

오랜만에 볼만한 한국 영화였어요!의 문장은81.46%로 긍정 리뷰입니다.


In [92]:
logi.predict_proba(vect_review)

array([[0.18543175, 0.81456825]])

In [96]:
# 데이터 프레임화 시키기
voc = pd.DataFrame(final_tfidf.vocabulary_.keys(),
                   index = final_tfidf.vocabulary_.values())
voc.sort_index()
# 트큰화 시 붙은 숫자 값은 국어사전 어순 순서대로 붙는 것을 확인 할 수 있다.

,0
0,ㄱ
1,ㄴ
2,ㄷ
3,ㄹ
4,ㅁ
...,...
8743,힘쓰
8744,힘없
8745,힘입
8746,힙합


In [100]:
# 가중치 확인하기
learning_result = pd.DataFrame(logi.coef_.T, index = voc.sort_index()[0],
                               columns = ['w'])
learning_result.sort_values(by = 'w')

,w
0,
재미없,-4.487693
최악,-4.267965
아깝,-4.098824
쓰레기,-4.055906
실망,-3.620450
...,...
좋,3.310072
수작,3.317169
재미있,3.716394
